In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import missingno as msno
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler



In [ ]:
df = pd.read_excel('Open_DATA_Import Janar-Dhjetor-2024.xlsx', sheet_name='IMPORT_1')
df.head()

# Dataset Description

| **Column**         | **Description**                                                                 |
|---------------------|---------------------------------------------------------------------------------|
| **VITI**           | Year of the transaction.                                                        |
| **MUAJI**          | Month of the transaction.                                                       |
| **Regjimi**        | Customs regime applied to the transaction.                                      |
| **Origjina**       | Country or region of origin of the goods.                                       |
| **Kodi Tarifor**   | Tariff code (HS code) classifying the goods.                                    |
| **Sasia**          | Quantity of goods traded.                                                       |
| **Vlera Mallrave** | Monetary value of the goods.                                                    |
| **Netweight**      | Net weight of the goods (excluding packaging).                                  |
| **Taksa Doganës**  | Customs duty applied to the goods.                                              |
| **Taksa Akcizës**  | Excise duty applied to specific goods (e.g., alcohol, tobacco).                 |
| **Taksa TVSH-së**  | Value Added Tax (VAT) applied to the goods.                                     |

# Data inspect

In [ ]:
print(df.info)

The dataset contains **233,638 entries (rows)** and **11 columns**.
The columns are of different data types:
- **Integer (int64):** `VITI`, `MUAJI`
- **Float (float64):** `Sasia`, `Vlera Mallrave`, `Netweight`, `Taksa Doganës`, `Taksa Akcizës`, `Taksa TVSH-së`
- **Object (string/categorical):** `Regjimi`, `Origjina`, `Kodi Tarifor`


In [ ]:
df.describe()

#### **Count:**
- All columns (`VITI`, `MUAJI`, `Sasia`, `Vlera Mallrave`, `Netweight`, `Taksa Doganës`, `Taksa Akcizës`, `Taksa TVSH-së`) have **233,638 non-null entries**, except for `Sasia`, which has **72,943 non-null entries**. This indicates that the `Sasia` column has a significant number of missing values.

---

#### **Mean:**
- **`VITI`:** The mean value is **2024.0**, indicating that all transactions occurred in the year 2024.
- **`MUAJI`:** The mean value is **6.63**, suggesting that the average month of transactions is around June/July.
- **`Sasia`:** The mean quantity of goods traded is **27,979.52**, indicating a large volume of goods on average.
- **`Vlera Mallrave`:** The mean value of goods is **27,086.81**, showing a high average monetary value per transaction.
- **`Netweight`:** The mean net weight of goods is **23,324.68**, indicating heavy shipments on average.
- **`Taksa Doganës`:** The mean customs duty is **719.66**, suggesting moderate tax rates on average.
- **`Taksa Akcizës`:** The mean excise duty is **2,621.70**, indicating higher taxes on specific goods (e.g., alcohol, tobacco).
- **`Taksa TVSH-së`:** The mean VAT is **4,395.68**, showing a significant tax burden on goods.

---

#### **Standard Deviation (std):**
- **`VITI`:** The standard deviation is **0.0**, confirming that all transactions occurred in the same year (2024).
- **`MUAJI`:** The standard deviation is **3.44**, indicating moderate variability in the month of transactions.
- **`Sasia`:** The standard deviation is **635,763.40**, showing extremely high variability in the quantity of goods traded.
- **`Vlera Mallrave`:** The standard deviation is **252,921.97**, indicating high variability in the value of goods.
- **`Netweight`:** The standard deviation is **440,532.40**, showing extremely high variability in the net weight of goods.
- **`Taksa Doganës`:** The standard deviation is **9,445.22**, indicating significant variability in customs duties.
- **`Taksa Akcizës`:** The standard deviation is **129,111.77**, showing extremely high variability in excise duties.
- **`Taksa TVSH-së`:** The standard deviation is **60,603.43**, indicating significant variability in VAT.

---

#### **Minimum (min):**
- **`VITI`:** The minimum value is **2024**, confirming that all transactions occurred in 2024.
- **`MUAJI`:** The minimum value is **1**, indicating transactions occurred as early as January.
- **`Sasia`:** The minimum quantity of goods traded is **0.01**, showing very small transactions.
- **`Vlera Mallrave`:** The minimum value of goods is **0.0**, indicating some transactions have no monetary value.
- **`Netweight`:** The minimum net weight is **0.005**, showing very light shipments.
- **`Taksa Doganës`:** The minimum customs duty is **0.0**, indicating some transactions are duty-free.
- **`Taksa Akcizës`:** The minimum excise duty is **0.0**, indicating some goods are exempt from excise tax.
- **`Taksa TVSH-së`:** The minimum VAT is **0.0**, indicating some transactions are VAT-free.

---

#### **Quartiles (25%, 50%, 75%):**
- **`MUAJI`:**
  - 25% of transactions occurred by **April**, 50% by **July**, and 75% by **October**.
- **`Sasia`:**
  - 25% of transactions involve **4.06** units, 50% involve **41.0** units, and 75% involve **593.92** units.
- **`Vlera Mallrave`:**
  - 25% of transactions have a value of **242.5**, 50% have **1,564.37**, and 75% have **9,675.28**.
- **`Netweight`:**
  - 25% of shipments weigh **10.0**, 50% weigh **132.0**, and 75% weigh **1,782.90**.
- **`Taksa Doganës`:**
  - 25% of transactions have **0.0** customs duty, 50% have **5.89**, and 75% have **99.61**.
- **`Taksa Akcizës`:**
  - 25% of transactions have **0.0** excise duty, 50% have **0.0**, and 75% have **0.0**, indicating that most transactions are exempt from excise tax.
- **`Taksa TVSH-së`:**
  - 25% of transactions have **27.98** VAT, 50% have **201.96**, and 75% have **1,306.48**.

---

#### **Maximum (max):**
- **`VITI`:** The maximum value is **2024**, confirming all transactions occurred in 2024.
- **`MUAJI`:** The maximum value is **12**, indicating transactions occurred as late as December.
- **`Sasia`:** The maximum quantity of goods traded is **63,154,738.5**, indicating extremely large transactions.
- **`Vlera Mallrave`:** The maximum value of goods is **37,721,159.11**, indicating extremely high-value transactions.
- **`Netweight`:** The maximum net weight is **51,457,733.0**, indicating extremely heavy shipments.
- **`Taksa Doganës`:** The maximum customs duty is **1,015,552.62**, indicating very high tax rates for some transactions.
- **`Taksa Akcizës`:** The maximum excise duty is **22,565,098.44**, indicating extremely high taxes on specific goods.
- **`Taksa TVSH-së`:** The maximum VAT is **10,849,397.86**, indicating very high VAT for some transactions.

In [ ]:
df.shape

- **Number of Rows (Observations)**: 233,638
- **Number of Columns (Features)**: 11

In [ ]:
print(df.describe().T)

In [ ]:
df.describe(include=['O'])


In [ ]:
duplicated=df.duplicated().sum()
print((duplicated/len(df))*100 , '%')

- **Number of Duplicated Rows**: `0`
- **Percentage of Duplicated Rows**: `0.0%`

In [ ]:
# Check missing values in numerical columns
numerical_missing = df.select_dtypes(include=['number']).isnull().sum()
print(numerical_missing)

In [ ]:
# Check missing values in categorical columns
categorical_missing = df.select_dtypes(include=['object', 'category']).isnull().sum()
print(categorical_missing)

In [ ]:
sns.heatmap(df.isnull())


In [ ]:
integer_columns = ['VITI', 'MUAJI']
for column in integer_columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=column, data=df)
    plt.title(f'Frequency of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
float_columns = ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']
for column in float_columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], bins=30, kde=True)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()


    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[column])
    plt.title(f'Boxplot of {column}')
    plt.xlabel(column)
    plt.show()

In [ ]:
albanian_to_ascii = {
   'ë': 'e', 'Ë': 'E',
    'ç': 'c', 'Ç': 'C',
    'ë': 'e', 'Ë': 'E',
}

def replace_albanian_chars(text):
    for alb_char, ascii_char in albanian_to_ascii.items():
        text = text.replace(alb_char, ascii_char)
    return text


df['Kodi Tarifor'] = df['Kodi Tarifor'].apply(replace_albanian_chars)
df.head()

In [ ]:
import matplotlib.pyplot as plt

# supports Unicode characters
plt.rcParams['font.family'] = 'DejaVu Sans'

In [ ]:
object_columns = ['Regjimi', 'Origjina']
for column in object_columns:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=column, data=df)
    plt.title(f'Frequency of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.show()

    category_counts = df[column].value_counts()
    category_percentages = df[column].value_counts(normalize=True) * 100
    print(f"Category Counts for {column}:")
    print(category_counts)
    print(f"\nCategory Percentages for {column}:")
    print(category_percentages)

In [ ]:
numeric_cols = ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']
df[numeric_cols] = df[numeric_cols].replace({',': ''}, regex=True).astype(float)

plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Sasia', y='Vlera Mallrave', data=df)
plt.title('Scatter Plot: Sasia vs. Vlera Mallrave')
plt.xlabel('Sasia')
plt.ylabel('Vlera Mallrave')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.violinplot(x='Origjina', y='Vlera Mallrave', data=df)
plt.title('Violin Plot: Origjina vs. Vlera Mallrave')
plt.xlabel('Origjina')
plt.ylabel('Vlera Mallrave')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
df.groupby('Origjina')['Vlera Mallrave'].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Total Trade Value by Country')
plt.xlabel('Country')
plt.ylabel('Total Value of Goods')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df.groupby('Kodi Tarifor')['Vlera Mallrave'].sum().sort_values(ascending=False).head(10).plot(kind='bar')
plt.title('Top 10 Tariff Codes by Trade Value')
plt.xlabel('Tariff Code')
plt.ylabel('Total Value of Goods')
plt.show()

In [ ]:
tax_columns = ['Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']
total_taxes = df[tax_columns].sum()
plt.figure(figsize=(8, 6))
total_taxes.plot(kind='bar', stacked=True)
plt.title('Total Tax Contributions')
plt.xlabel('Tax Type')
plt.ylabel('Total Tax Amount')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
correlation_matrix = df[['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa TVSH-së']].corr()
print(correlation_matrix)


## Correlation Analysis Summary

1. Sasia and Netweight (0.801509):
   - Strong Positive Correlation: As the quantity (Sasia) increases, the net weight (Netweight) of goods also tends to increase. Larger quantities of goods typically have higher net weights.

2. Sasia and Vlera Mallrave (0.659401):
   - Moderate Positive Correlation: Higher quantities are associated with higher values of goods, which is expected.

3. Sasia and Taksa TVSH-së (0.719401):
   - Strong Positive Correlation: As the quantity increases, the VAT (Taksa TVSH-së) also increases. This is logical since VAT is usually calculated as a percentage of the value of goods, which correlates with quantity.

4. Sasia and Taksa Doganës (0.001626):
   - Very Weak Correlation: Almost no relationship between quantity and customs duty (Taksa Doganës), suggesting that customs duty is likely dependent on other factors (e.g., type of goods, origin).

5. Vlera Mallrave and Taksa TVSH-së (0.882032):
   - Very Strong Positive Correlation: A high correlation between the value of goods (Vlera Mallrave) and VAT (Taksa TVSH-së). This is expected as VAT is generally calculated as a percentage of the value of goods.

6. Vlera Mallrave and Netweight (0.631587):
   - Moderate Positive Correlation: Higher values of goods are associated with higher net weights, suggesting that more valuable goods might also be heavier or larger in quantity.


In [ ]:
df_notnull = df[df['Sasia'].notnull()]
df_null = df[df['Sasia'].isnull()]

X = df_notnull[['Netweight', 'Vlera Mallrave', 'Taksa TVSH-së']]
y = df_notnull['Sasia']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = RandomForestRegressor(random_state=42)
model.fit(X_scaled, y)

X_null = df_null[['Netweight', 'Vlera Mallrave', 'Taksa TVSH-së']]
X_null_scaled = scaler.transform(X_null)
df_null.loc[:, 'Sasia'] = model.predict(X_null_scaled)

df = pd.concat([df_notnull, df_null])
print("Negative values in 'Sasia':", df[df['Sasia'] < 0].shape[0])

In [ ]:
missing_percent = df.isnull().mean().sort_values(ascending=False) * 100
print("Missing Value Percentage by Columns:\n", round(missing_percent,2))

In [ ]:
print(df[['Sasia', 'Netweight', 'Vlera Mallrave', 'Taksa TVSH-së']].head(30))


In [ ]:
constant_columns = df.columns[df.nunique() == 1]
print("Constant columns:", constant_columns)

In [ ]:
df.drop(columns=constant_columns, inplace=True)

In [ ]:
df['Origjina'] = df['Origjina'].astype('category')
df['Kodi Tarifor'] = df['Kodi Tarifor'].astype('category')

In [ ]:
def detect_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return outliers


for col in ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']:
    outliers = detect_outliers_iqr(df[col])
    print(f"Përjashtuesit në {col}:\n", outliers)

In [ ]:
for col in ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))

In [ ]:
for col in ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Përjashtuesit pas trajtimit në {col}:\n", outliers)

In [ ]:
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
skewness = df[numerical_columns].skew()
print("Koeficienti i anueshmërisë:\n", skewness)

In [ ]:
for col in ['Sasia', 'Vlera Mallrave', 'Netweight', 'Taksa Doganës', 'Taksa Akcizës', 'Taksa TVSH-së']:
    df[col + '_log'] = np.log1p(df[col])

print("\nKoeficienti i anueshmërisë pas transformimit logaritmik:")
print(df[['Sasia_log', 'Vlera Mallrave_log', 'Netweight_log', 'Taksa Doganës_log', 'Taksa Akcizës_log', 'Taksa TVSH-së_log']].skew())

In [ ]:
df['Origjina'] = df['Origjina'].str.split(' - ').str[1]


In [ ]:
df.head()

In [ ]:
df['Kodi Tarifor'] = df['Kodi Tarifor'].str.split(' - ').str[0]

In [ ]:
df.head()

In [ ]:
missing_values = df.isnull().sum()
total_dataset = np.prod(df.shape)
total_missing = missing_values.sum()
percent = (total_missing / total_dataset) * 100
percent

In [ ]:
numeric_columns1 = df.select_dtypes(include=[np.number]).columns

for column in numeric_columns1:
    plt.figure(figsize=(14, 7))

    # Subplot 1: Kernel Density Estimation Plot
    plt.subplot(1, 2, 1)
    plt.title(f"{column} - Almost Normal Distribution", fontsize=15)
    sns.kdeplot(data=df[column], fill=True, color="blue", warn_singular=False)

    # Subplot 2: Box Plot
    plt.subplot(1, 2, 2)
    plt.title(f"{column} - Skewed Distribution", fontsize=15)
    sns.boxplot(data=df[column], orient="h")

    plt.tight_layout()
    plt.show()

In [ ]:
# Histogram for data distribution before cleaning
df['Netweight'].hist(bins=20)
plt.title('Histogram of Netweight')
plt.xlabel('Netweight')
plt.ylabel('Frequency')
plt.show()

# Histogram for data distribution after cleaning
df['Netweight_log'].hist(bins=20)
plt.title('Histogram of Netweight')
plt.xlabel('Netweight')
plt.ylabel('Frequency')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)